In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
summer_refractive_index = 1+0.00020409512542851616
winter_refractive_index = 1+0.00020953240736654166
import copy 
import sys
import glob
from datetime import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.interpolate import interp1d
import statsmodels.api as sm

import matplotlib.pyplot as plt
from ctapipe.io import EventSource
from ctapipe.image import ImageProcessor, tailcuts_clean
from ctapipe.calib import CameraCalibrator
from ctapipe.visualization import CameraDisplay
from astropy.table import Table, vstack

#from muonpipe import usefull_func
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
filename = '/Users/vdk/muons2024/simtel_files/2024year_tuned_nooulier_reflectivity_additional/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 1)
camgeom = source.subarray.tel[1].camera.geometry

start_date_2019 = datetime.strptime("2019-11-25 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2020 = datetime.strptime("2020-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

size_outside_cut = 500
muon_efficiency_cut = 1
min_impact = 2.2199933748101555
max_impact = 9.983608702234397


# Standard simulation with nsb and reflectivity tuned to 2024 year

In [ ]:
listdir1 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity/tables/*')
listdir2 = glob.glob('/Users/vdk/muons2024/lapalma_simulations/for_paper/nsbtune2024year_no_outliers_additional_true/tables/*')
listdir = listdir1 + listdir2

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')
    
    # Check if 'good_ring' exists and convert to bool
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_00043_alignment = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]
df_sim_00043_alignment

# Simulations 0.015 deg mirror random align

In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_0015alignment/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_0015_alignment = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]

df_sim_0015_alignment

## Starting height of particle 3200m

In [ ]:
listdir= glob.glob('/Users/vdk/muons2024/psf_work/fixchi710/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_0015_alignment_3200m = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]

df_sim_0015_alignment_3200m

## Sim height=3200m Broken Power Law and 6-80 GeV

In [ ]:
listdir1 = glob.glob('/Users/vdk/muons2024/psf_work/fixchi710/broken_spectrum/tables_6-10gev/*')
listdir2 = glob.glob('/Users/vdk/muons2024/psf_work/fixchi710/tables_10-20gev/*')
listdir3 = glob.glob('/Users/vdk/muons2024/psf_work/fixchi710/tables_20-80gev/*')

listdir = listdir1 + listdir2 + listdir3

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_broken = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]

df_sim_broken

# Sim starting height = 2700

In [ ]:
listdir = glob.glob('/Users/vdk/muons2024/psf_work/fixchi753/tables/*')

dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_0015_alignment_2700m = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]

df_sim_0015_alignment_2700m

# Data good quality winter 2024, zenith angle < 20 deg

In [ ]:
winter_good_data_2024 = glob.glob('/Users/vdk/muons2024/data_quality_winter_2024/tables/*')

dat = Table.read('{}'.format(winter_good_data_2024[0]), format='fits')

for muon_file in winter_good_data_2024:
    dat2 = Table.read(muon_file, format='fits')

    if 'good_ring' in dat.colnames:
        dat['good_ring'] = dat['good_ring'].astype(bool)
    if 'is_valid' in dat2.colnames:
        dat2['is_valid'] = dat2['is_valid'].astype(bool)
    if 'parameters_at_limit' in dat2.colnames:
        dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)       
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()

df_good_data_winter_2024 = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]

print()
min_event_time = datetime.fromtimestamp(df_good_data_winter_2024['event_time'].min()).strftime('%d-%m-%Y')
max_event_time = datetime.fromtimestamp(df_good_data_winter_2024['event_time'].max()).strftime('%d-%m-%Y')
print(f"Min event time: {min_event_time}, Max event time: {max_event_time}")

df_good_data_winter_2024

# Applying additional cuts

In [ ]:
ring_completeness_cut = 0
ring_containment_cut = 0
min_radius_cut = 0
max_radius_cut = 2
ring_center_distance_cut = 3
impact_distance_cut = 15

def calculate_survival_percentage(df, df_cut):
    return (len(df_cut) / len(df)) * 100

def apply_cuts(df, completeness_cut, containment_cut, min_radius_cut, max_radius_cut, center_distance_cut, impact_distance_cut):
    return df[
        (df['ring_completeness'] > completeness_cut) &
        (df['ring_containment'] > containment_cut) &
        (df['ring_radius'] > min_radius_cut) &
        (df['ring_radius'] < max_radius_cut) &
        (df['ring_center_y'].abs() < center_distance_cut) &
        (df['impact_parameter'] < impact_distance_cut)
    ]


# Apply cuts to each dataframe
# df_data_cut = apply_cuts(df_good_data_real, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
# print(f"Events survived for df_data_cut: {len(df_data_cut)} ({calculate_survival_percentage(df_good_data_real, df_data_cut):.2f}%) out of {len(df_good_data_real)}")

df_good_data_winter_2024_cut = apply_cuts(df_good_data_winter_2024, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_good_data_winter_2024_cut: {len(df_good_data_winter_2024_cut)} ({calculate_survival_percentage(df_good_data_winter_2024, df_good_data_winter_2024_cut):.2f}%) out of {len(df_good_data_winter_2024)}")

df_sim_00043_cut = apply_cuts(df_sim_00043_alignment, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_00043_cut: {len(df_sim_00043_cut)} ({calculate_survival_percentage(df_sim_00043_alignment, df_sim_00043_cut):.2f}%) out of {len(df_sim_00043_alignment)}")

df_sim_0015_cut = apply_cuts(df_sim_0015_alignment, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_0015_cut: {len(df_sim_0015_cut)} ({calculate_survival_percentage(df_sim_0015_alignment, df_sim_0015_cut):.2f}%) out of {len(df_sim_0015_alignment)}")

df_sim_0015_cut_3200m = apply_cuts(df_sim_0015_alignment_3200m, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_0015_alignment_3200m_cut: {len(df_sim_0015_cut_3200m)} ({calculate_survival_percentage(df_sim_0015_alignment_3200m, df_sim_0015_cut_3200m):.2f}%) out of {len(df_sim_0015_alignment_3200m)}")

df_sim_0015_cut_2700m = apply_cuts(df_sim_0015_alignment_2700m, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_0015_alignment_2700m_cut: {len(df_sim_0015_cut_2700m)} ({calculate_survival_percentage(df_sim_0015_alignment_2700m, df_sim_0015_cut_2700m):.2f}%) out of {len(df_sim_0015_alignment_2700m)}")

df_sim_broken_cut = apply_cuts(df_sim_broken, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_broken_cut: {len(df_sim_broken_cut)} ({calculate_survival_percentage(df_sim_broken, df_sim_broken_cut):.2f}%) out of {len(df_sim_broken)}")

# Plotting

In [ ]:

# List of dataframes to plot
dfs = {
    'Good data winter 2024' : df_good_data_winter_2024_cut,
    'Standard sim' : df_sim_00043_cut,
    'Mirror random align 0.015 deg' : df_sim_0015_cut,
    'Sim with start height 3200' :df_sim_0015_cut_3200m,
    'Sim with start height 2700' : df_sim_0015_cut_2700m,
    '6-80 GeV broken PL' : df_sim_broken_cut
}

# Plot histograms for each dataframe
plt.figure(figsize=(15, 10))

for labels, df in dfs.items():
    df['ring_radius'].hist(bins=50, alpha=0.5, label=labels, density=True, histtype='step', lw = 3)

plt.title('Histograms of ring_radius for all DataFrames', fontsize=20)
plt.xlabel('Ring radius [deg]', fontsize=15)
plt.ylabel('Frequency', fontsize=15)
plt.legend()
plt.show()

# Reweighting

In [ ]:
def weighted_data(df, power_index):
    df_copy = copy.deepcopy(df[['ring_radius', 'mc_energy', 'ring_completeness', 'ring_containment', 'impact_parameter']])
    df_copy['weights'] = df_copy['mc_energy'] ** (-power_index)
    return df_copy

def calculate_weights_broken_power_law(df, energy_breaks, power_indices):
    """
    Calculate weights using a broken power law.
    
    Parameters:
    - df (pd.DataFrame): DataFrame with at least 'mc_energy' column.
    - energy_breaks (list): List of energy breakpoints for the broken power law.
    - power_indices (list): List of power indices for each energy segment.
    
    Returns:
    - pd.DataFrame: A DataFrame with weights added.
    """
    if len(energy_breaks) + 1 != len(power_indices):
        raise ValueError("The number of power indices must be one more than the number of energy breaks.")
    
    df_copy = df.copy()
    df_copy['weights'] = np.nan

    # Assign weights based on the energy ranges
    for i, power_index in enumerate(power_indices):
        if i == 0:
            mask = df_copy['mc_energy'] < energy_breaks[0]
            print(f"LEN MASK {sum(mask)}")
        elif i == len(power_indices) - 1:
            mask = df_copy['mc_energy'] >= energy_breaks[-1]
            print(f"LEN MASK {sum(mask)}")
        else:
            mask = (df_copy['mc_energy'] >= energy_breaks[i - 1]) & (df_copy['mc_energy'] < energy_breaks[i])
            print(f"LEN MASK {sum(mask)}")
        
        df_copy.loc[mask, 'weights'] = df_copy.loc[mask, 'mc_energy'] ** (-power_index)

    return df_copy


In [ ]:
df_sim_standard_power_law_weigth = weighted_data(df_sim_0015_cut, 0.7)

df_sim_0015_cut_3200m_weight = weighted_data(df_sim_0015_cut_3200m, 0.7)

df_sim_0015_cut_2700m_weight = weighted_data(df_sim_0015_cut_2700m, 0.7)

#df_sim_broken_power_law_weight_016 = weighted_data(df_sim_0015_cut[df_sim_0015_cut['mc_energy'] < 0.01], 0.12)
#df_sim_broken_power_law_weight_046 = weighted_data(df_sim_0015_cut[(df_sim_0015_cut['mc_energy'] > 0.01) & (df_sim_0015_cut['mc_energy'] < 0.02)], 0.46)
#df_sim_broken_power_law_weight_046 = weighted_data(df_sim_0015_cut[(df_sim_0015_cut['mc_energy'] < 0.02)], 0.46)
#df_sim_broken_power_law_weight_071 = weighted_data(df_sim_0015_cut[df_sim_0015_cut['mc_energy'] >= 0.02], 0.71)

#df_sim_broken_power_law_weight_all = pd.concat([df_sim_broken_power_law_weight_046, df_sim_broken_power_law_weight_071], ignore_index=True)

In [ ]:
dfs = {
    'Mirror random align 0.015 deg (weighted)' : df_sim_standard_power_law_weigth,
    'Sim with start height 3200 (weighted)' :df_sim_0015_cut_3200m_weight,
}

plt.figure(figsize=(15, 10))

for labels, df in dfs.items():
    df['ring_radius'].hist(bins=50, alpha=0.5, label=labels, density=True, histtype='step', lw = 3, weights=df['weights'])

df_good_data_winter_2024_cut['ring_radius'].hist(bins=50, alpha=0.5, label='Good data winter 2024', density=True, histtype='step', lw = 3)

plt.title('Histograms of ring_radius for all DataFrames', fontsize=20)
plt.xlabel('Ring radius [deg]', fontsize=15)
plt.ylabel('Frequency', fontsize=15)
plt.legend()
plt.show()